In [1]:
import warnings
warnings.filterwarnings('ignore')
import pandas as pd

In [2]:
import numpy as np
import matplotlib.pyplot as plt

In [3]:
from sklearn.preprocessing import Imputer
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import StandardScaler

In [4]:
from sklearn.decomposition import PCA

In [5]:
dataset = pd.read_csv('train.csv')

In [6]:
dataset.head()

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,...,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
0,1,60,RL,65.0,8450,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,2,2008,WD,Normal,208500
1,2,20,RL,80.0,9600,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,5,2007,WD,Normal,181500
2,3,60,RL,68.0,11250,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,9,2008,WD,Normal,223500
3,4,70,RL,60.0,9550,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,2,2006,WD,Abnorml,140000
4,5,60,RL,84.0,14260,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,12,2008,WD,Normal,250000


In [7]:
x_train = dataset.iloc[:,1:-1]

In [8]:
y = dataset.iloc[:,-1].values

In [9]:
# returns discrete and contiuous features

def differentiate_features(x_train):
    discrete_features = []
    continuous_features = []
    for i,j in zip(x_train,x_train.dtypes):
        if j.type==np.int64 or j.type==np.float64:
            continuous_features.append(i)
        else:
            discrete_features.append(i)
    return discrete_features,continuous_features

In [11]:
discrete_features,continuous_features = differentiate_features(x_train)

In [12]:
discrete_features[:5],continuous_features[:5]

(['MSZoning', 'Street', 'Alley', 'LotShape', 'LandContour'],
 ['MSSubClass', 'LotFrontage', 'LotArea', 'OverallQual', 'OverallCond'])

In [13]:
len(discrete_features),len(continuous_features),len(discrete_features)+len(continuous_features)

(43, 36, 79)

In [14]:
# replaces NaN values in continuous features to MEAN value
def get_imputer(x_continuous):
    imputer = Imputer(missing_values = np.nan, strategy = 'mean', axis = 0)
    imputer.fit(x_continuous)
    return imputer

In [15]:
x_continuous = dataset.filter(continuous_features).values
imputer = get_imputer(x_continuous)

C:\Users\sappusamy\AppData\Local\Continuum\anaconda3\envs\sri\lib\site-packages\sklearn\utils\deprecation.py:58: DeprecationWarning: Class Imputer is deprecated; Imputer was deprecated in version 0.20 and will be removed in 0.22. Import impute.SimpleImputer from sklearn instead.
  warnings.warn(msg, category=DeprecationWarning)


In [16]:
def get_data_from_imputer(imputer,x_continuous):
    x_continuous = imputer.transform(x_continuous)
    return x_continuous

In [17]:
x_continuous = get_data_from_imputer(imputer,x_continuous)

In [18]:
x_continuous.shape

(1460, 36)

In [19]:
# creates object of LABEL ENCODER for each discrete varible
# each unique categories are replaced with LABEL(index like 0,1,2...)
def get_label_encoder(discrete_features,x_discrete):
    label_encoder_dict={}
    for col in discrete_features:
        label_encoder_dict[col] = LabelEncoder()
        x_discrete[col] = x_discrete[col].fillna('')
        s = x_discrete[col].astype(str).unique()
        if '' not in s: s = np.append(s,[''])
        label_encoder_dict[col].fit(s)
    return label_encoder_dict

In [20]:
x_discrete = dataset.filter(discrete_features)

In [21]:
label_encoder_dict = get_label_encoder(discrete_features,x_discrete)

In [22]:
len(label_encoder_dict)

43

In [23]:
def get_discrete_encoder(label_encoder_dict,x_discrete):
    for col in label_encoder_dict:
        x_discrete[col] = label_encoder_dict[col].transform(x_discrete[col])
    return x_discrete

In [24]:
x_discrete = get_discrete_encoder(label_encoder_dict,x_discrete)

In [25]:
x_discrete.head()

,MSZoning,Street,Alley,LotShape,LandContour,Utilities,LotConfig,LandSlope,Neighborhood,Condition1,...,GarageType,GarageFinish,GarageQual,GarageCond,PavedDrive,PoolQC,Fence,MiscFeature,SaleType,SaleCondition
0,4,2,0,4,4,1,5,1,6,3,...,2,2,5,5,3,0,0,0,9,5
1,4,2,0,4,4,1,3,1,25,2,...,2,2,5,5,3,0,0,0,9,5
2,4,2,0,1,4,1,5,1,6,3,...,2,2,5,5,3,0,0,0,9,5
3,4,2,0,1,4,1,1,1,7,3,...,6,3,5,5,3,0,0,0,9,1
4,4,2,0,1,4,1,3,1,16,3,...,2,2,5,5,3,0,0,0,9,5


In [26]:
try:
    n_discrete = x_discrete.shape[1]
except:
    n_discrete = 0
n_discrete

43

In [27]:
# ONE HOT ENCODER for feature i.e., [Male,Female]->[0,1] (Label encoder output)
# which means 1 column expanded to 2 columns
def generate_onehot_encoder(n_discrete):
    onehotencoder = OneHotEncoder(categorical_features = list(range(n_discrete)))
    onehotencoder.fit(x_discrete)
    return onehotencoder

In [28]:
onehotencoder = generate_onehot_encoder(n_discrete)

C:\Users\sappusamy\AppData\Local\Continuum\anaconda3\envs\sri\lib\site-packages\sklearn\preprocessing\_encoders.py:392: DeprecationWarning: The 'categorical_features' keyword is deprecated in version 0.20 and will be removed in 0.22. You can use the ColumnTransformer instead.
  "use the ColumnTransformer instead.", DeprecationWarning)


In [29]:
# returns 2 things
# 1 - output of one hot encoder
# 2 - concatenation of output of discrete and continuous features
def get_onehot_encoder(onehotencoder,x_continuous,x_discrete):
    x_discrete = onehotencoder.transform(x_discrete).toarray()
    x = np.concatenate([x_continuous,x_discrete],axis=1)
    return x,x_discrete

In [30]:
x,x_discrete = get_onehot_encoder(onehotencoder,x_continuous,x_discrete)

In [31]:
x_discrete,x_discrete.shape

(array([[0., 0., 0., ..., 0., 1., 0.],
        [0., 0., 0., ..., 0., 1., 0.],
        [0., 0., 0., ..., 0., 1., 0.],
        ...,
        [0., 0., 0., ..., 0., 1., 0.],
        [0., 0., 0., ..., 0., 1., 0.],
        [0., 0., 0., ..., 0., 1., 0.]]), (1460, 268))

In [32]:
x.shape

(1460, 304)

In [33]:
# 43 discrete features has been expanded to 268 features
# 36 - continuous features
# totally 268+36=304 features

In [34]:
# standarize the input features
# x = (x-mean(x))/std_deviation(x)

def get_standardscaler(x,y):
    sc_X = StandardScaler()
    sc_X.fit(x)
    sc_y = StandardScaler()
    sc_y.fit(y.reshape(-1,1))
    return sc_X,sc_y

In [35]:
sc_X,sc_Y = get_standardscaler(x,y)

In [36]:
def get_normalized(sc_X,sc_Y,x,y):
    x = sc_X.transform(x)
    y = sc_Y.transform(y.reshape(-1,1))
    return x,y.reshape(-1)

In [37]:
x_train,y_train = get_normalized(sc_X,sc_Y,x,y)

In [38]:
y_train

array([ 0.34727322,  0.00728832,  0.53615372, ...,  1.07761115,
       -0.48852299, -0.42084081])

In [39]:
x_train = np.concatenate([np.ones((len(x_train),1)),x_train],axis=1)

In [40]:
num_features = x_train.shape[1]
num_features

305

In [41]:
#root mean square error
def rmse(preds,actual):
    return ((preds-actual)**2).sum()/(2*len(preds))

In [42]:
#cost function with regularization
def loss_fn(preds,actual,weight):
    return ((preds-actual)**2).sum()/(2*len(preds)) + (rp*np.square(weight).sum())

In [43]:
#optimize function - derivative with regularization
def optimize(y,y_train,x_train,lr,rp,weight):
    weight[0] = weight[0]-lr*np.matmul((y-y_train),x_train[:,0])/len(x_train)
    weight[1:] = (weight[1:]*(1-(lr*rp/len(x_train)))) - lr*np.matmul((y-y_train),x_train[:,1:])/len(x_train)
    return weight

In [ ]:
epochs = 100
w = np.random.uniform(low=-0.1,high=0.1,size=(num_features))
# w = np.random.randn((num_features))*np.sqrt(2/(1+num_features))
lr = 1e-1
rp = 1e-4

In [ ]:
loss=0
for epoch in range(epochs):
    pred_y = np.matmul(x_train,w)
    loss = loss_fn(pred_y,y_train,w)
    print('Epoch ',epoch,':-',loss)
    w = optimize(pred_y,y_train,x_train,lr,rp,w)

In [ ]:
x_train.shape

In [ ]:
def get_normalized_test_data(x_test,
                             y_test,
                             continuous_features,
                             discrete_features,
                             imputer,
                             label_encoder_dict,
                             onehotencoder,
                             sc_X,
                             sc_Y
                            ):
    x_test_continuous = x_test.filter(continuous_features)
    x_test_discrete = x_test.filter(discrete_features)
    
    x_test_continuous = get_data_from_imputer(imputer,x_test_continuous)
    
    x_test_discrete = x_test_discrete.fillna('')
    
    x_test_discrete = get_discrete_encoder(label_encoder_dict,x_test_discrete)
    
    x,_ = get_onehot_encoder(onehotencoder,x_test_continuous,x_test_discrete)
    
    x_test,y_test = get_normalized(sc_X,sc_Y,x,y_test)
    
    x_test = np.concatenate([np.ones((len(x_test),1)),x_test],axis=1)
    
    return x_test,y_test
    

In [ ]:
x_test = pd.read_csv('test.csv')

In [ ]:
y_test = pd.read_csv('sample_submission.csv')

In [ ]:
y_test = y_test['SalePrice'].to_numpy()

In [ ]:
x_test, y_test = get_normalized_test_data(x_test,
                             y_test,
                             continuous_features,
                             discrete_features,
                             imputer,
                             label_encoder_dict,
                             onehotencoder,
                             sc_X,
                             sc_Y
                            )

In [ ]:
pred_y = np.matmul(x_test,w)

In [ ]:
rmse(pred_y,y_test)

In [ ]:
# plot of test data points
plt.scatter(x_test.mean(1),y_test,color='k',label='data points')
# plt.plot(x_test.mean(1),pred_y,'r',label='test',linewidth=2)

plt.title('Data points')
plt.xlabel('Input')
plt.ylabel('Output')

plt.legend()

plt.grid(False)

plt.show()

In [ ]:
# plot of test data points
# plt.scatter(x_test.mean(1),y_test,color='k',label='data points')
plt.plot(x_test.mean(1),pred_y,'r',label='test',linewidth=2)

plt.title('Prediction curve')
plt.xlabel('Input')
plt.ylabel('Output')

plt.legend()

plt.grid(False)

plt.show()

In [ ]:
# plot of test data points
plt.scatter(x_test.mean(1),y_test,color='k',label='data points')
plt.plot(x_test.mean(1),pred_y,'r',label='test',linewidth=2)

plt.title('Prediction curve for test data points')
plt.xlabel('Input')
plt.ylabel('Output')

plt.legend()

plt.grid(False)

plt.show()